## Precarga de librerias y funciones

In [1]:
%run "../recurrentes.ipynb"

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [2]:
%run "../funciones.ipynb"

In [3]:
# # Preseteo optuna especificar bucket y nombre del estudio


# nombre_archivo = 'optimization_tree.db'
# bucket = 'b2/'


# estudio_optuna = base_path + 'buckets/' + bucket + 'optimization_tree.db'


# # cargar estudio
# # a) competencia_02
# # b) competencia_02_lags
# # c) competencia_02_lags_y_deltas

# nombre_estudio = 'competencia_02_lags'


#### Excel

In [4]:
# Activar Excel de seguimiento de medidas? 

excel = True
if excel == True:
    # Ingresar path excel para anotar resultados
    path_excel = base_path_l + r'\resultados_backtesting.xlsx'
    date = datetime.now().strftime("%d-%m-%Y %H-%M-%S")
    

In [5]:
# 2 - Cargar datos
# Opciones:
# a) dataset_clase_ternaria_l
# b) dataset_lags_clase_ternaria_l
# c) dataset_lags_deltas_y_clase_ternaria_l


dataset = dataset_lags_deltas_y_clase_ternaria_l
dataset_name = os.path.basename(dataset)
df_train = pd.read_parquet(dataset)


## Formateo pre modelo

In [84]:
if 'T_Visa_normal' in df_train.columns:
    df_train['T_Visa_normal'] = df_train['T_Visa_normal'].astype(bool)
if 'T_Master_normal'in df_train.columns:
    df_train['T_Master_normal'] = df_train['T_Master_normal'].astype(bool)


In [ ]:
# 3 - parametros para modelo
ganancia_acierto = 273000
costo_estimulo = 7000

mes_train = 202104
mes_test = 202106


In [86]:
data = df_train
data.shape

(4735593, 600)

In [87]:

data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [88]:
data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)

In [ ]:
# Especificar mes de train y test

df_train = data[data['foto_mes']<=mes_train]
df_test = data[data['foto_mes']==mes_test]


In [90]:
print(data.shape)  # Número de filas y columnas
print(data.info(memory_usage='deep'))

(4735593, 602)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4735593 entries, 0 to 4735592
Columns: 602 entries, numero_de_cliente to clase_binaria
dtypes: bool(2), float64(535), int32(1), int64(63), object(1)
memory usage: 21.4 GB
None


In [91]:
clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']

In [92]:
w_test = df_test.loc[Y_test.index, 'clase_peso']
test_data = lgb.Dataset(X_test, label=Y_test, weight=w_test)
test_data

In [93]:
w_train = df_train.loc[X_train.index, 'clase_peso']


## Cross-Validation

In [16]:
# Ordenar entradas en df_train en base a la columna foto_mes de meses más antiguos a más recientes
X_train = X_train.sort_values('foto_mes', ascending=True)

In [20]:
semillas

[400009, 500009, 500011, 500021, 600009]

In [22]:
tscv = TimeSeriesSplit(n_splits=5)
folds = tscv.split(X_train)

lista_cv = []

params = {
    'objective': 'binary',  # Puedes cambiar esto si tu problema es multiclase u otro tipo
    'metric': 'binary_logloss',  # Cambia el metric si es necesario
    }

train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)




# Pasar las divisiones a lgb.cv
cv_results = lgb.cv(
    params,
    train_data,
    num_boost_round=300,
    feval=lgb_gan_eval,
    folds=folds,  # Aquí especificamos las divisiones temporales
    seed=s,
)

cv_results

max_gan = max(cv_results['valid gan_eval-mean'])
best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

final =  max_gan * 5
lista_cv.append(final)



[LightGBM] [Info] Number of positive: 7210, number of negative: 672040
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.071578 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 679250, number of used features: 154
[LightGBM] [Info] Number of positive: 76290, number of negative: 1282210
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.134004 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 1358500, number of used features: 154
[LightGBM] [Info] Number of positive: 294796, number of negative: 1742954
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhea

In [27]:
tscv = TimeSeriesSplit(n_splits=5)
folds = tscv.split(X_train)


params = {
    'objective': 'binary',  # Puedes cambiar esto si tu problema es multiclase u otro tipo
    'metric': 'binary_logloss',  # Cambia el metric si es necesario
    }

train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)




# Pasar las divisiones a lgb.cv
cv_results = lgb.cv(
    params,
    train_data,
    num_boost_round=300,
    feval=lgb_gan_eval,
    folds=folds,  # Aquí especificamos las divisiones temporales
    seed=semillas[1],
)

cv_results

max_gan = max(cv_results['valid gan_eval-mean'])
best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

final_1 =  max_gan * 5
lista_cv.append(final_1)

[LightGBM] [Info] Number of positive: 7210, number of negative: 672040
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.089561 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 679250, number of used features: 154
[LightGBM] [Info] Number of positive: 76290, number of negative: 1282210
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.130294 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 1358500, number of used features: 154
[LightGBM] [Info] Number of positive: 294796, number of negative: 1742954
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhea

In [23]:
tscv = TimeSeriesSplit(n_splits=5)
folds = tscv.split(X_train)


params = {
    'objective': 'binary',  # Puedes cambiar esto si tu problema es multiclase u otro tipo
    'metric': 'binary_logloss',  # Cambia el metric si es necesario
    }

train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)




# Pasar las divisiones a lgb.cv
cv_results = lgb.cv(
    params,
    train_data,
    num_boost_round=300,
    feval=lgb_gan_eval,
    folds=folds,  # Aquí especificamos las divisiones temporales
    seed=semillas[2],
)

cv_results

max_gan = max(cv_results['valid gan_eval-mean'])
best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

final_2 =  max_gan * 5
lista_cv.append(final_2)

[LightGBM] [Info] Number of positive: 7210, number of negative: 672040
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.062586 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 679250, number of used features: 154
[LightGBM] [Info] Number of positive: 76290, number of negative: 1282210
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.129290 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 1358500, number of used features: 154
[LightGBM] [Info] Number of positive: 294796, number of negative: 1742954
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhea

In [24]:
tscv = TimeSeriesSplit(n_splits=5)
folds = tscv.split(X_train)




params = {
    'objective': 'binary',  # Puedes cambiar esto si tu problema es multiclase u otro tipo
    'metric': 'binary_logloss',  # Cambia el metric si es necesario
    }

train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)




# Pasar las divisiones a lgb.cv
cv_results = lgb.cv(
    params,
    train_data,
    num_boost_round=300,
    feval=lgb_gan_eval,
    folds=folds,  # Aquí especificamos las divisiones temporales
    seed=semillas[3],
)

cv_results

max_gan = max(cv_results['valid gan_eval-mean'])
best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

final_3 =  max_gan * 5
lista_cv.append(final_3)

[LightGBM] [Info] Number of positive: 7210, number of negative: 672040
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.074897 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 679250, number of used features: 154
[LightGBM] [Info] Number of positive: 76290, number of negative: 1282210
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.156746 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 1358500, number of used features: 154
[LightGBM] [Info] Number of positive: 294796, number of negative: 1742954
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhea

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
folds = tscv.split(X_train)



params = {
    'objective': 'binary',  # Puedes cambiar esto si tu problema es multiclase u otro tipo
    'metric': 'binary_logloss',  # Cambia el metric si es necesario
    }

train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)




# Pasar las divisiones a lgb.cv
cv_results = lgb.cv(
    params,
    train_data,
    num_boost_round=300,
    feval=lgb_gan_eval,
    folds=folds,  # Aquí especificamos las divisiones temporales
    seed=semillas[4],
)

cv_results

max_gan = max(cv_results['valid gan_eval-mean'])
best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

final_4 =  max_gan * 5
lista_cv.append(final_4)

[LightGBM] [Info] Number of positive: 7210, number of negative: 672040
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.074043 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 679250, number of used features: 154
[LightGBM] [Info] Number of positive: 76290, number of negative: 1282210
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.440371 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 22081
[LightGBM] [Info] Number of data points in the train set: 1358500, number of used features: 154
[LightGBM] [Info] Number of positive: 294796, number of negative: 1742954
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.191630 seconds.
You can set `force_row_wise=t

In [28]:
finalisimo =  sum(lista_cv)/ len(lista_cv)  
finalisimo

10737406704.0

In [ ]:
# RESULTADOS:
# 1. Modelo Base = 10737406704.0

## Predicción y scoring

In [94]:
lista_medidas = [accuracy_score, precision_score, recall_score, f1_score, roc_auc_score]



In [95]:
train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)


In [ ]:
#Modelo utilizado para back testing utilizando parametros default si la diferencia en CV es mínima intentar optuna y luego 

params = {
    'objective': 'binary',  
    'metric': 'binary_logloss',  
    'seed': semillas[0], }
model_default = lgb.train(params, train_data)    
y_pred_default = model_default.predict(X_test)


#### Medidas

In [ ]:
dicc_medidas = {}
if excel == True:
    dicc_medidas['fecha'] = date
    dicc_medidas['dataset'] = dataset_name
    

In [ ]:
y_pred_labels = (y_pred_default >= 0.025).astype(int)

In [ ]:
for medida in lista_medidas:
     dicc_medidas[medida.__name__] = medida(Y_test, y_pred_labels)
print(dicc_medidas)

{'fecha': '18-11-2024-', 'dataset': 'competencia_02_lags_y_clase_ternaria.parquet', 'ganancia': ('gan_eval', 83524000, True), 'accuracy_score': 0.9229966762900604, 'precision_score': 0.09640461294942339, 'recall_score': 0.6436839456467036, 'f1_score': 0.16769371968008392, 'roc_auc_score': 0.7850439078834234}


In [ ]:
ganancia = lgb_gan_eval(y_pred_default, test_data)
dicc_medidas['ganancia'] = ganancia[1]

In [ ]:
cantidad_columnas = df_train.shape[1]
mes_train = df_train['foto_mes'].max()
mes_test = df_test['foto_mes'].max()

dicc_medidas['cantidad_columnas'] = cantidad_columnas
dicc_medidas['mes_train'] = mes_train
dicc_medidas['mes_test'] = mes_test

In [ ]:
dicc_medidas

{'fecha': '18-11-2024-',
 'dataset': 'competencia_02_lags_y_clase_ternaria.parquet',
 'ganancia': 83524000,
 'accuracy_score': 0.9229966762900604,
 'precision_score': 0.09640461294942339,
 'recall_score': 0.6436839456467036,
 'f1_score': 0.16769371968008392,
 'roc_auc_score': 0.7850439078834234}

In [ ]:
if excel == True:
    
    wb = load_workbook(path_excel)
    ws = wb.active

    header_row = 1  
    column_mapping = {cell.value: cell.column for cell in ws[header_row]}

    # Verificar el mapeo de columnas
    print("Mapeo de columnas:", column_mapping)
    
row_to_update = None

for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=1):
    cell = row[0]  # La primera celda de la fila
    if cell.value is None:  # Si el valor de la celda está vacía
        print(f"Primera celda vacía en la fila {cell.row}")
        row_to_update = cell.row
        break

# Si no se encontró ninguna celda vacía, agregar al final
if row_to_update is None:
    print("No se encontro celda vacía, agregando al final")
    row_to_update = ws.max_row + 1  # Agregar en una nueva fila al final

print(f"Fila a actualizar: {row_to_update}")
fila_actualizar = row_to_update  # Fila donde escribir los datos

for col_name, value in dicc_medidas.items():
    if col_name in column_mapping:  # Asegurarse de que la columna existe
        col_idx = column_mapping[col_name]
        ws.cell(row=fila_actualizar, column=col_idx, value=value)

# Guardar los cambios
wb.save(path_excel)



Mapeo de columnas: {'fecha': 1, 'dataset': 2, 'ganancia': 3, 'accuracy_score': 4, 'precision_score': 5, 'recall_score': 6, 'f1_score': 7, 'roc_auc_score': 8}
Fila a actualizar: 2
